# 크롤링 시도하기

In [1]:
from bs4 import BeautifulSoup as bs
import requests
from fake_useragent import UserAgent
import json
import pandas as pd
import traceback

In [2]:
ua = UserAgent(verify_ssl=False)
fake_ua = ua.random
headers = {
    'user-agent' : fake_ua
}

In [3]:
BASE_URL = 'https://gift.kakao.com/' # 기본 URL bs
product_n = '1238682' # 상품ID  # 1 ~ 7009001
URL2 = 'https://gift.kakao.com/product/{}'.format(product_n) # 상품페이지 bs
PRODUCT_INFO_URL = 'https://gift.kakao.com/a/v2/products/{}?'.format(product_n) # 상품정보/ 가격, 브랜드, 카테고리 js
WISH_URL = 'https://gift.kakao.com/a/v1/items/{}/wishes/wish?'.format(product_n) # 찜하기수 js
STAT_URL = 'https://gift.kakao.com/a/v1/review/products/{}/stat?'.format(product_n) # 리뷰수, 만족도 js
REVIEW_URL = 'https://gift.kakao.com/a/v2/review/products/{}/attachments?page=0&size=48&'.format(product_n) # 리뷰 js

In [ ]:
# df_rank.colunm = ['rank','name','brand','category','price','wish','review_count','rating'] # 랭킹상품데이터
# df.colunm = ['name','brand','category','price','wish','review_count','rating'] # 상품데이터

In [7]:
def product_info(pd_idlist: list)  -> pd.DataFrame : 
    """ 
    Product_id : list -> Product_info : DataFrame \n
    colunm = ['name', 'brand', 'category', 'price', 'wish', 'review_count', 'rating'] 
    """

    # BASE_URL = 'https://gift.kakao.com/' # 기본 URL bs
    # URL2 = 'https://gift.kakao.com/product/{}'.format(pd_idlist) # 상품페이지 bs
    # REVIEW_URL = 'https://gift.kakao.com/a/v2/review/products/{}/attachments?page=0&size=48&'.format(pd_idlist) # 리뷰 js
    
    product_df = pd.DataFrame(columns=['id','name','brand','category','price','wish','review_count','rating'])

    for id in pd_idlist :
        try:
            PRODUCT_INFO_URL = 'https://gift.kakao.com/a/v2/products/{}?'.format(id) # 상품정보/ 가격, 브랜드, 카테고리 js
            WISH_URL = 'https://gift.kakao.com/a/v1/items/{}/wishes/wish?'.format(id) # 찜하기수 js
            STAT_URL = 'https://gift.kakao.com/a/v1/review/products/{}/stat?'.format(id) # 리뷰수, 만족도 js

        
            json_p = json.loads(requests.get(PRODUCT_INFO_URL).text) # name, brand, category, price
            json_w = json.loads(requests.get(WISH_URL).text) # wish
            json_s = json.loads(requests.get(STAT_URL).text) # review_count , rating

            name = json_p['itemDetails']['item']['name'] # name
            brand = json_p['itemDetails']['brand']['name'] # brand
            category = json_p['itemDetails']['category']['name'] # category
            price = json_p['itemDetails']['item']['sellingPrice'] # price

            wish = json_w['wishCount'] # wish

            review_count = json_s['totalCount'] # review_count
            rating1 = json_s['productRating1Percent'] # rating 1
            rating2 = json_s['productRating2Percent'] # rating 2
            rating3 = json_s['productRating3Percent'] # rating 3
            rating4 = json_s['productRating4Percent'] # rating 4

            rating = rating1/100 * 1 + rating2/100 * 2 + rating3/100 * 3 + rating4/100 * 4 # rating_avg

            df_list = [id, name, brand, category, price, wish, review_count, rating] # tolist

            product_df.loc[len(product_df)] = df_list 
    
        except:
            traceback.print_exc()
            with open('error_urls', 'a') as f:
                f.write(str(id)+'\n')

    return product_df

In [8]:
product_info([138682])

,id,name,brand,category,price,wish,review_count,rating
0,138682,메이빈/참 순수한 캡 10팩(80매),메이빈,기저귀/물티슈,18000,0,0,0.0


In [ ]:
sample = [
	{
		"group": "delivery/4",
		"ranking": 1,
		"id": 5259781
	},
	{
		"group": "delivery/4",
		"ranking": 2,
		"id": 2370524
	},
	{
		"group": "delivery/4",
		"ranking": 3,
		"id": 5023078
	}]

In [ ]:
id_list = [ dic['id'] for dic in sample ]

In [ ]:
print(id_list)

In [23]:
with open('./product_id2_all.json', 'r') as f:

    json_data = json.load(f)

In [24]:
id_list = [ dic['id'] for dic in json_data ]

In [25]:
print(id_list)

[3818674, 2255190, 2255187, 2272985, 3818648, 3818654, 3818623, 3747561, 2272982, 5820227, 1138952, 5804311, 3818639, 3747516, 6508161, 5022500, 4668406, 6508318, 6847470, 6223678, 3818518, 3818522, 6171995, 1138942, 1307384, 6806246, 3818571, 4702432, 4059021, 6224939, 6147422, 3818398, 4677746, 4058936, 1244879, 1160874, 3754423, 1244795, 2255186, 420270, 1307395, 6408298, 1244882, 6508435, 3818212, 4165956, 4638731, 5804125, 5641536, 547128, 6806438, 5601353, 4059008, 3818185, 2272969, 4676981, 4702425, 6862426, 5022360, 3818616, 6806241, 3789963, 3789952, 5535197, 420246, 3818209, 4979017, 6508429, 3746443, 6782208, 3968206, 6495676, 6408285, 6913698, 5601279, 6863736, 5345441, 4059025, 4165696, 5022414, 547122, 4058937, 6851561, 4165964, 5022358, 1244798, 2442676, 6542267, 5022407, 420272, 3749415, 1138956, 6193867, 6346294, 2272988, 5543372, 4058933, 6868773, 4595915, 3248562]


In [26]:
results = product_info(id_list) # 2천개 상품 id -> 상품정보 -> 13분

In [27]:
results

,id,name,brand,category,price,wish,review_count,rating
0,3818674,아이스 카페 아메리카노T 2잔+부드러운 생크림 카스테라,스타벅스,카페,13500,19778,184955,3.90
1,2255190,e카드 3만원 교환권,스타벅스 상품권,카페,30000,88714,109148,3.92
2,2255187,e카드 5만원 교환권,스타벅스 상품권,카페,50000,30423,51666,3.92
3,2272985,"신세계상품권 모바일교환권 100,000원","이마트,신세계상품권",상품권,100000,25133,35574,3.88
4,3818648,아이스 카페 아메리카노 T+7 레이어 가나슈 케이크,스타벅스,카페,10200,16217,86754,3.90
...,...,...,...,...,...,...,...,...
95,5543372,그릭요거트 딸기 생크림 피스 + 아메리카노 (R),투썸플레이스,카페,10700,4632,1072,3.89
96,4058933,카카오페이상품권 3만원권(교환권),카카오페이상품권,상품권,30000,9332,918,3.86
97,6868773,광동)비타500 100ML+롯데)가나초콜릿마일드1200,GS25,편의점,2200,496,61,3.98
98,4595915,5만원권,투썸플레이스,카페,50000,2215,1993,3.88


In [28]:
results.to_csv('ranking_product_2_all.csv')

In [29]:
df_1 = pd.read_json('product_id2_all.json')

In [30]:
df_1.drop(['id'], axis=1, inplace=True)

In [31]:
df_2 = pd.read_csv('ranking_product_2_all.csv')

In [32]:
df_2.drop(['Unnamed: 0'], axis=1, inplace=True)

In [33]:
df_2

,id,name,brand,category,price,wish,review_count,rating
0,3818674,아이스 카페 아메리카노T 2잔+부드러운 생크림 카스테라,스타벅스,카페,13500,19778,184955,3.90
1,2255190,e카드 3만원 교환권,스타벅스 상품권,카페,30000,88714,109148,3.92
2,2255187,e카드 5만원 교환권,스타벅스 상품권,카페,50000,30423,51666,3.92
3,2272985,"신세계상품권 모바일교환권 100,000원","이마트,신세계상품권",상품권,100000,25133,35574,3.88
4,3818648,아이스 카페 아메리카노 T+7 레이어 가나슈 케이크,스타벅스,카페,10200,16217,86754,3.90
...,...,...,...,...,...,...,...,...
95,5543372,그릭요거트 딸기 생크림 피스 + 아메리카노 (R),투썸플레이스,카페,10700,4632,1072,3.89
96,4058933,카카오페이상품권 3만원권(교환권),카카오페이상품권,상품권,30000,9332,918,3.86
97,6868773,광동)비타500 100ML+롯데)가나초콜릿마일드1200,GS25,편의점,2200,496,61,3.98
98,4595915,5만원권,투썸플레이스,카페,50000,2215,1993,3.88


In [34]:
df_3 = pd.concat([df_1,df_2],axis=1)

In [35]:
df_3.to_csv('ranking_product_2_all_concat.csv')

In [36]:
df_3

,group,ranking,id,name,brand,category,price,wish,review_count,rating
0,best/coupon,1,3818674,아이스 카페 아메리카노T 2잔+부드러운 생크림 카스테라,스타벅스,카페,13500,19778,184955,3.90
1,best/coupon,2,2255190,e카드 3만원 교환권,스타벅스 상품권,카페,30000,88714,109148,3.92
2,best/coupon,3,2255187,e카드 5만원 교환권,스타벅스 상품권,카페,50000,30423,51666,3.92
3,best/coupon,4,2272985,"신세계상품권 모바일교환권 100,000원","이마트,신세계상품권",상품권,100000,25133,35574,3.88
4,best/coupon,5,3818648,아이스 카페 아메리카노 T+7 레이어 가나슈 케이크,스타벅스,카페,10200,16217,86754,3.90
...,...,...,...,...,...,...,...,...,...,...
95,best/coupon,96,5543372,그릭요거트 딸기 생크림 피스 + 아메리카노 (R),투썸플레이스,카페,10700,4632,1072,3.89
96,best/coupon,97,4058933,카카오페이상품권 3만원권(교환권),카카오페이상품권,상품권,30000,9332,918,3.86
97,best/coupon,98,6868773,광동)비타500 100ML+롯데)가나초콜릿마일드1200,GS25,편의점,2200,496,61,3.98
98,best/coupon,99,4595915,5만원권,투썸플레이스,카페,50000,2215,1993,3.88
